In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/mnist-data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /mnist-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /mnist-data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /mnist-data/t10k-images-idx3-ubyte.gz
Extracting /mnist-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 定义输入输出
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [4]:
# 第一层卷积
# 28*28*1 -> 14*14*32
W_conv1 = tf.get_variable('W_conv1', shape=[5, 5, 1, 32], initializer=tf.truncated_normal_initializer(stddev=0.1))
b_conv1 = tf.get_variable('b_conv1', shape=[32], initializer=tf.constant_initializer(0.1))

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# 第二层卷积
# 14*14*32 -> 7*7*64
W_conv2 = tf.get_variable('W_conv2', shape=[5, 5, 32, 64], initializer=tf.truncated_normal_initializer(stddev=0.1))
b_conv2 = tf.get_variable('b_conv2', shape=[64], initializer=tf.constant_initializer(0.1))

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [6]:
# 第三层全连接
W_fc1 = tf.get_variable('W_fc1', shape=[7 * 7 * 64, 1024], initializer=tf.truncated_normal_initializer(stddev=0.1))
b_fc1 = tf.get_variable('b_fc1', shape=[1024], initializer=tf.constant_initializer(0.1))

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 并加入Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# 第四层输出层
W_fc2 = tf.get_variable('W_fc2', shape=[1024, 10], initializer=tf.truncated_normal_initializer(stddev=0.1))
b_fc2 = tf.get_variable('b_fc2', shape=[10], initializer=tf.constant_initializer(0.1))

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [0]:
# 算法优化与模型评估
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_op = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
for i in range(1600):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    sess.run(train_op, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%sess.run(accuracy, feed_dict={
        x:mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.1
step 100, training accuracy 0.84
step 200, training accuracy 0.9
step 300, training accuracy 0.94
step 400, training accuracy 0.88
step 500, training accuracy 0.98
step 600, training accuracy 0.94
step 700, training accuracy 0.98
step 800, training accuracy 0.96
step 900, training accuracy 0.96
step 1000, training accuracy 1
step 1100, training accuracy 0.98
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 0.96
step 1500, training accuracy 0.94
test accuracy 0.9706
